1. Data Scraping

We collect the top *n* movies sorted by year with the following criteria:
- Have atleast 1000 reviews
- Is a Movie (Not a series)
- Is in English

In [48]:
import os
import pandas as pd
import requests, zipfile, io

In [25]:
notebooks = [1,2,3,4,5]

#Run the web scraping notebooks if the csv data doesnt exist
for notebook in notebooks:
    notebook_pth = f'new_features_only_{notebook}.ipynb'
    data = f'raw{notebook}.csv'
    n = f'test.ipynb'

    if not os.path.exists(data):
        %run $notebook_pth

In [31]:
#concatenate every data
full_data = pd.read_csv('raw1.csv')

for notebook in range(2, 5+1):
    data = f'raw{notebook}.csv'
    partial_data = pd.read_csv(data)
    full_data = pd.concat([full_data,partial_data], ignore_index=True)

In [70]:
full_data.head()

,Unnamed: 0,Title,Year,Plot,Rating,Budget,Cast,MPA Rating,Directors,Distributors,Producers,Genre,Runtime in Minutes
0,0,1. Whiplash,2014,A promising young drummer enrolls at a cut-thr...,8.5,"$3,300,000 (estimated)","Miles Teller,J.K. Simmons,Paul Reiser,Melissa ...",TV-14,Damien Chazelle,"Sony Pictures Classics,Sony Pictures Worldwide...","Bold Films,Blumhouse Productions,Right of Way ...","Drama,Music",106
1,1,2. RoboCop,2014,"In 2028 Detroit, when Alex Murphy, a loving hu...",6.1,"$100,000,000 (estimated)","Joel Kinnaman,Gary Oldman,Michael Keaton,Abbie...",TV-14,José Padilha,"Columbia Pictures,ACME,ACME,ACME,B&H Film Dist...","Metro-Goldwyn-Mayer (MGM),Columbia Pictures,St...","Action,Crime,Sci-Fi,Thriller",117
2,2,3. The Babadook,2014,A single mother and her child fall into a deep...,6.8,"$2,000,000 (estimated)","Essie Davis,Noah Wiseman,Hayley McElhinney,Dan...",Not,Jennifer Kent,"Capelight Pictures,Cinecolor Films,Feelgood En...","Screen Australia,Causeway Films,The South Aust...","Drama,Horror,Mystery",94
3,3,4. What We Do in the Shadows,2014,"Viago, Deacon, and Vladislav are vampires who ...",7.6,"$1,600,000 (estimated)","Jemaine Clement,Taika Waititi,Jonny Brugh,Cori...",R,"Jemaine Clement,Taika Waititi","Benuca Films,Cinetren,Festival Films / Yedra F...","Unison Films,Defender Films,Park Road Post Pro...","Comedy,Horror",86
4,4,5. Boyhood,2014,"The life of Mason, from early childhood to his...",7.9,"$4,000,000 (estimated)","Ellar Coltrane,Patricia Arquette,Elijah Smith,...",TV-14,Richard Linklater,"Diaphana Distribution,IFC Films,Lumière,Lumièr...","IFC Productions,Detour Filmproduction",Drama,165


In [34]:
full_data.to_csv("raw_complete.csv")
print("DONE compiling")

DONE compiling


In [63]:
if not os.path.exists('deflators.csv'):
    deflators = requests.get("https://api.worldbank.org/v2/en/indicator/NY.GDP.DEFL.ZS?downloadformat=csv", allow_redirects=True)
    unzipped = zipfile.ZipFile(io.BytesIO(deflators.content))
    file = 'API_NY.GDP.DEFL.ZS_DS2_en_csv_v2_22116.csv'

    with unzipped.open(file) as f:
        df_deflators = pd.read_csv(f, skiprows=4)
    usd_row = df_deflators[df_deflators['Country Code'] == 'USA']
    usd_row.to_csv('deflators.csv')